In [2]:
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
import pandas as pd
from sklearn.metrics import make_scorer, fbeta_score
from sklearn.model_selection import StratifiedKFold

train_data_loaded = pd.read_csv('../data/train_data_2024-08-01.csv')

X = train_data_loaded.drop(columns=['UKATEGORIE'])
y = train_data_loaded['UKATEGORIE']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# KFold-Konfiguration
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Definieren des F-beta-Scores mit beta = 2
beta = 2
fbeta_scorer = make_scorer(fbeta_score, beta=beta)

knn_param_space = {
    'n_neighbors': Integer(5, 50),
    'leaf_size': Integer(20, 50),
    'p': Integer(1, 2),
    'weights': Categorical(['uniform', 'distance'])
}

# Anwendung von SMOTE auf Trainingsset
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_train, y_train)

# K-Nearest Neighbors
knn_opt = BayesSearchCV(estimator=KNeighborsClassifier(), search_spaces=knn_param_space, n_iter=32, cv=kf, n_jobs=-1, scoring=fbeta_scorer, random_state=42)
knn_opt.fit(X_res, y_res)
print("Best parameters for KNN: ", knn_opt.best_params_)

# Best parameters from BayesSearchCV
best_knn_params = knn_opt.best_params_

# Erstelle das KNN Modell mit den besten Parametern
knn_best = KNeighborsClassifier(**best_knn_params)

# Führe die Kreuzvalidierung durch mit Resampling innerhalb der Kreuzvalidierung
scores = cross_val_score(knn_best, X_res, y_res, cv=kf, scoring=fbeta_scorer, n_jobs=-1)

# Ausgabe der Ergebnisse
print(f"Kreuzvalidierungsergebnisse: {scores}")
print(f"Durchschnittlicher Wert: {scores.mean()} ± {scores.std()}")

print("Best fbeta_score for KNN: ", knn_opt.best_score_)

# Auch nachfolgendes Cross-Validation führt man besser auf den resampleten Daten durch
# Perform Cross-Validation auf Trainingsdaten
train_scores = cross_val_score(knn_best, X_res, y_res, cv=kf, scoring=fbeta_scorer)

# Anwendung von SMOTE auf Testset
X_test_res, y_test_res = sm.fit_resample(X_test, y_test)

# Perform Cross-Validation auf Testdaten
test_scores = cross_val_score(knn_best, X_test_res, y_test_res, cv=kf, scoring=fbeta_scorer)

print("Cross-Validation Scores für Trainingsdaten: ", train_scores)
print("Mean Cross-Validation Score für Trainingsdaten: ", train_scores.mean())
print("Standard Deviation Cross-Validation Score für Trainingsdaten: ", train_scores.std())

print("\nCross-Validation Scores für Testdaten: ", test_scores)
print("Mean Cross-Validation Score für Testdaten: ", test_scores.mean())
print("Standard Deviation Cross-Validation Score für Testdaten: ", test_scores.std())

/home/kasch/Nextcloud/IKT/venv/lib/python3.12/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [50, 5, 1, 'distance'] before, using random point [42, 48, 2, 'distance']
  warnings.warn(


Best parameters for KNN:  OrderedDict({'leaf_size': 41, 'n_neighbors': 6, 'p': 1, 'weights': 'distance'})
Kreuzvalidierungsergebnisse: [0.90690743 0.90934586 0.90786193 0.91141367 0.90697267]
Durchschnittlicher Wert: 0.9085003135587506 ± 0.0017018280173942426
Best fbeta_score for KNN:  0.9085003135587506
Cross-Validation Scores für Trainingsdaten:  [0.90690743 0.90934586 0.90786193 0.91141367 0.90697267]
Mean Cross-Validation Score für Trainingsdaten:  0.9085003135587506
Standard Deviation Cross-Validation Score für Trainingsdaten:  0.0017018280173942426

Cross-Validation Scores für Testdaten:  [0.91709772 0.92219423 0.91427365 0.91866835 0.92556454]
Mean Cross-Validation Score für Testdaten:  0.9195596990659244
Standard Deviation Cross-Validation Score für Testdaten:  0.003944541495031128
